In [1]:
from source.inputters.corpus import KnowledgeCorpus
corpus = KnowledgeCorpus('./data/', 'demo', max_len=500, max_vocab_size=30000, share_vocab=True)
corpus.load()
valid_raw = corpus.read_data('./data/demo.dev', data_type='valid')
valid_data = corpus.build_examples(valid_raw)

from source.inputters.dataset import Dataset
data = Dataset(valid_data)
loader = data.create_batches(5)
for input in loader:
    input = input
    break

Loading prepared vocab from ./data/demo_30000.vocab.pt ...
Vocabulary size of fields: SRC-30004 TGT-30004 CUE-30004
Loading prepared data from ./data/demo_30000.data.pt ...
Number of examples: TRAIN-89901 VALID-9054 TEST-9054
Read 9054 VALID examples (0 filtered)


100%|████████████████████████████████████████████████████████████████████████████| 9054/9054 [00:00<00:00, 9846.95it/s]


In [2]:
corpus.SRC.stoi['瓦莱莉']

10033

In [3]:
corpus.SRC.itos[3]

'<eos>'

In [4]:
enc_inputs = input
# y(t-1)
dec_inputs = input.tgt[0][:, :-1], input.tgt[1] - 1
# y(t)
target = input.tgt[0][:, 1:]

In [5]:
enc_inputs.src[0][:, 1:-1]

tensor([[  30,    4,    8,    8,   36,    4,   29,   96,    3,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [  30,    4,    8,    8,   36,    4,   29,   45,   26,   50,   11,   37,
          292,   45,   26,   50,   11,   10,    3,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [  30,    4,    8,    8,   36,    4,   29,   45,   26,   50,   11,   37,
          292,   

In [6]:
from source.models.pgnet import PointerNet
model = PointerNet(corpus.SRC.vocab_size, 300, 800, padding_idx=corpus.padding_idx)

In [7]:
# prob_vocab, prob_hist, prob_fact, p_modes = model(enc_inputs, dec_inputs)

In [8]:
# outputs, dec_init_state = model.encode(enc_inputs, None)

In [9]:
dec_init_state.fact_mask[0]

NameError: name 'dec_init_state' is not defined

In [ ]:
dec_init_state.fact[0]

In [ ]:
import torch

In [ ]:
dec_inputs, _ = dec_inputs

In [ ]:
dec_inputs[0]

In [ ]:
input = dec_inputs[:, 0]

In [ ]:
input

In [ ]:
# prob_vocab, p_mode, attn_f, attn_h, fact, hist = model.decoder.decode(input, dec_init_state)

In [ ]:
weighted_prob = prob_vocab * p_mode[:, :, 0].unsqueeze(1)
weighted_f = attn_f * p_mode[:, :, 1].unsqueeze(1)
weighted_h = attn_h * p_mode[:, :, 2].unsqueeze(1)

In [ ]:
from source.utils.misc import convert_dist

In [ ]:
weighted_prob = convert_dist(
                weighted_h, hist, weighted_prob)

In [ ]:
weighted_prob = convert_dist(
                weighted_f, fact, weighted_prob)

In [ ]:
attn_f

In [ ]:
prob_vocab

In [ ]:
# log_prob, state, output = model.decoder.decode(input, dec_init_state)

In [ ]:
# out_facts, fact, hist, out_hists, prob_vocab, prob_hist, prob_fact, p_modes = model(enc_inputs, dec_inputs)

In [10]:
model(enc_inputs, dec_inputs)

{'logits': tensor([[[-11.6557,  -5.6015, -11.8224,  ..., -11.6369, -11.6120, -11.4678],
          [-11.6246,  -5.3138, -11.5654,  ..., -11.6774, -11.4716, -11.5826],
          [-11.5315,  -5.3819, -11.3849,  ..., -11.4009, -11.2214, -11.2855],
          ...,
          [-11.3874, -11.4205, -11.4199,  ..., -11.4226, -11.4029, -11.3950],
          [-11.3874, -11.4205, -11.4199,  ..., -11.4226, -11.4029, -11.3950],
          [-11.3874, -11.4205, -11.4199,  ..., -11.4226, -11.4029, -11.3950]],
 
         [[-11.5824,  -5.6269, -11.7955,  ..., -11.5905, -11.6226, -11.4493],
          [-11.3700,  -5.4281, -11.4291,  ..., -11.4367, -11.3609, -11.2757],
          [-11.7680,  -5.2559, -11.7753,  ..., -11.8492, -11.8443, -11.7730],
          ...,
          [-11.3874, -11.4205, -11.4199,  ..., -11.4226, -11.4029, -11.3950],
          [-11.3874, -11.4205, -11.4199,  ..., -11.4226, -11.4029, -11.3950],
          [-11.3874, -11.4205, -11.4199,  ..., -11.4226, -11.4029, -11.3950]],
 
         [[-11.613

In [ ]:
fact[0]

In [ ]:
prob_fact[0][0][3]

In [ ]:
import torch
a = torch.cat((prob_vocab, prob_hist, prob_fact), -
                      1).view(5*24, 30004, -1)
b = p_modes.view(5 * 24, -1).unsqueeze(2)

In [ ]:
prob = torch.bmm(a, b).squeeze().view(5,24, -1)

In [ ]:
prob.size()

In [ ]:
prob